# TensorFlow - Embeddings Viz

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

### Get IMdb data

In [ ]:
# 50,000 movie reviews labeled as "positive" or "negative"
imdb_data = pd.read_csv("IMDB Dataset.csv")

In [ ]:
imdb_data["sentiment"] = imdb_data["sentiment"].map({"positive": 1, "negative":0})

In [ ]:
imdb_data[:5]

### Train/Test Split

In [ ]:
imdb_train = imdb_data["review"].values[:25000]
imdb_test = imdb_data["review"].values[25000:]

imdb_train_labels = imdb_data["sentiment"].values[:25000]
imdb_test_labels = imdb_data["sentiment"].values[25000:]

In [ ]:
# unconverted sentence

imdb_train[0]

In [ ]:
imdb_train_labels[0]

### Tokenize

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer(num_words=10000, oov_token = "<oov>")
tokenizer.fit_on_texts(imdb_train)
word_index = tokenizer.word_index

# convert words to numbers and pad for the neural network to use as input
# Training data
train_sequences = tokenizer.texts_to_sequences(imdb_train)
train_padded = pad_sequences(train_sequences, maxlen=120, truncating="post")

# Test data
testing_sequences = tokenizer.texts_to_sequences(imdb_test)
test_padded = pad_sequences(testing_sequences, maxlen=120, truncating="post")

In [ ]:
# sentence converted to sequence

train_sequences[0]

### Compare Tokenization

In [ ]:
# Reverses keys: keys become the values, and values become the keys so that we can look a word up
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

# This is what will be fed in
print(decode_review(train_padded[0]))

# This is the original text
print(imdb_train[0])

### Create Embeddings

In [ ]:
vocab_size = 10000
embedding_dim = 16

model = tf.keras.Sequential([
    # The Embedding layer is the key to text sentiment analysis
    tf.keras.layers.Embedding(10000, 16, input_length=120),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_padded, imdb_train_labels, epochs=10, validation_split=.1)

### Embeddings

In [ ]:
# Output from the Embedding layer
embeddings = model.layers[0]

weights = embeddings.get_weights()[0]
print(f"Vocabulary size: {weights.shape[0]},  Embedding dimensions: {weights.shape[1]}")

In [ ]:
weights

### Visualize the embedding vectors

In [ ]:
import io

# writing the vectors and their metadata out to file. 
# these 2 files ('vecs.tsv', 'meta.tsv') are used by the 
# TensorFlow projector (http://projector.tensorflow.org/)
# to plot/visualize the vectors/embeddings in 3D

# Output of the 16 values per word representation (embedding)
#      out_v are the weights (embedding)
#      out_m are the actual words associated with each embedding

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()